In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import boto3


%matplotlib inline

import os
print(os.listdir())

import warnings
warnings.filterwarnings('ignore')

['lost+found', 'Untitled1 (1).ipynb', '.virtual_documents', '.ipynb_checkpoints', '.sparkmagic']


In [8]:
s3 = boto3.client('s3')
bucket = "team-transform-newdata"
key = "run-1744970736858-part-r-00000"
local_file = "/tmp/run-1744970736858-part-r-00000"

# Download the file
s3.download_file(bucket, key, local_file)
print("Download complete.")

Download complete.


In [9]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd

dataset = pd.read_csv(local_file)
print(dataset.head())

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  target  
0   0     1       1  
1   0     2       1  
2   0     2       1  
3   0     2       1  
4   0     2       1  


In [11]:
type(dataset)

pandas.core.frame.DataFrame

In [12]:
dataset.shape

(303, 14)

In [13]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [14]:
info = ["age","1: male, 0: female","chest pain type, 1: typical angina, 2: atypical angina, 3: non-anginal pain, 4: asymptomatic","resting blood pressure"," serum cholestoral in mg/dl","fasting blood sugar > 120 mg/dl","resting electrocardiographic results (values 0,1,2)"," maximum heart rate achieved","exercise induced angina","oldpeak = ST depression induced by exercise relative to rest","the slope of the peak exercise ST segment","number of major vessels (0-3) colored by flourosopy","thal: 3 = normal; 6 = fixed defect; 7 = reversable defect"]


for i in range(len(info)):
    print(dataset.columns[i]+":\t\t\t"+info[i])

age:			age
sex:			1: male, 0: female
cp:			chest pain type, 1: typical angina, 2: atypical angina, 3: non-anginal pain, 4: asymptomatic
trestbps:			resting blood pressure
chol:			 serum cholestoral in mg/dl
fbs:			fasting blood sugar > 120 mg/dl
restecg:			resting electrocardiographic results (values 0,1,2)
thalach:			 maximum heart rate achieved
exang:			exercise induced angina
oldpeak:			oldpeak = ST depression induced by exercise relative to rest
slope:			the slope of the peak exercise ST segment
ca:			number of major vessels (0-3) colored by flourosopy
thal:			thal: 3 = normal; 6 = fixed defect; 7 = reversable defect


In [15]:
dataset["target"].describe()

count    303.000000
mean       0.544554
std        0.498835
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: target, dtype: float64

In [16]:
dataset["target"].unique()

array([1, 0])

In [17]:
from sklearn.model_selection import train_test_split

predictors = dataset.drop("target",axis=1)
target = dataset["target"]

X_train,X_test,Y_train,Y_test = train_test_split(predictors,target,test_size=0.20,random_state=0)

In [18]:
X_train.shape

(242, 13)

In [19]:
X_test.shape

(61, 13)

In [20]:
Y_train.shape

(242,)

In [21]:
Y_test.shape

(61,)

### Random Forest

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
from sklearn.ensemble import RandomForestClassifier

max_accuracy = 0


for x in range(2000):
    rf = RandomForestClassifier(random_state=x)
    rf.fit(X_train,Y_train)
    Y_pred_rf = rf.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_rf,Y_test)*100,2)
    if(current_accuracy>max_accuracy):
        max_accuracy = current_accuracy
        best_x = x
        
#print(max_accuracy)
#print(best_x)

rf = RandomForestClassifier(random_state=best_x)
rf.fit(X_train,Y_train)
Y_pred_rf = rf.predict(X_test)

In [24]:
Y_pred_rf.shape

(61,)

In [25]:
score_rf = round(accuracy_score(Y_pred_rf,Y_test)*100,2)

print("The accuracy score achieved using Decision Tree is: "+str(score_rf)+" %")

The accuracy score achieved using Decision Tree is: 90.16 %


In [26]:
# Sample user input (can be from a form, API, etc.)
user_input = {
    "age": 54,
    "sex": 1,  # 1 = male, 0 = female
    "cp": 2,   # chest pain type
    "trestbps": 120,
    "chol": 250,
    "fbs": 0,
    "restecg": 2,
    "thalach": 140,
    "exang": 0,
    "oldpeak": 1.4,
    "slope": 2,
    "ca": 0,
    "thal": 3
}

# Convert to DataFrame for prediction
input_df = pd.DataFrame([user_input])

# Predict
prediction = rf.predict(input_df)[0]

# Output the result
print("Prediction:", prediction)
if prediction == 1:
    print("⚠️  Risk of heart disease detected.")
else:
    print("✅ No heart disease detected.")


Prediction: 1
⚠️  Risk of heart disease detected.


In [27]:
user_input = {
    "age": 45,          # Younger age
    "sex": 0,           # Female (typically lower risk in some datasets)
    "cp": 1,            # Typical angina (lower-risk chest pain type)
    "trestbps": 120,    # Normal resting blood pressure
    "chol": 180,        # Lower cholesterol
    "fbs": 0,           # Fasting blood sugar <= 120
    "restecg": 0,       # Normal ECG
    "thalach": 175,     # High max heart rate achieved (good)
    "exang": 0,         # No exercise-induced angina
    "oldpeak": 0.0,     # No ST depression
    "slope": 1,         # Up-sloping ST segment
    "ca": 0,            # No major vessels blocked
    "thal": 3           # Normal thalassemia
}

# Convert to DataFrame and predict
input_df = pd.DataFrame([user_input])
prediction = rf.predict(input_df)[0]

print("Prediction:", prediction)
if prediction == 1:
    print("⚠️  Risk of heart disease detected.")
else:
    print("✅ No heart disease detected.")


Prediction: 1
⚠️  Risk of heart disease detected.


In [28]:
user_input = {
    "age": 30,          # Younger age
    "sex": 1,           # Male (could slightly increase risk, but not necessarily)
    "cp": 0,            # No chest pain (no angina)
    "trestbps": 110,    # Normal resting blood pressure (a bit lower, indicating good health)
    "chol": 160,        # Lower cholesterol level
    "fbs": 0,           # Fasting blood sugar <= 120
    "restecg": 0,       # Normal ECG
    "thalach": 180,     # High max heart rate achieved (good indicator)
    "exang": 0,         # No exercise-induced angina
    "oldpeak": 0.0,     # No ST depression
    "slope": 1,         # Up-sloping ST segment (normal)
    "ca": 0,            # No major vessels blocked
    "thal": 3           # Normal thalassemia
}

# Convert to DataFrame and predict
input_df = pd.DataFrame([user_input])
prediction = rf.predict(input_df)[0]

print("Prediction:", prediction)
if prediction == 1:
    print("⚠️  Risk of heart disease detected.")
else:
    print("✅ No heart disease detected.")


Prediction: 0
✅ No heart disease detected.


In [29]:
import joblib

# Save the trained model
joblib.dump(rf, 'random_forest_model.pkl')

['random_forest_model.pkl']